## Load Data and Tokenizer

In [1]:
from transformers import BertTokenizer
from datasets import load_dataset

ds = load_dataset('billingsmoore/tagged-tibetan-to-english-translation-dataset')

tokenizer = BertTokenizer.from_pretrained('tibetan_tokenizer')

## Preprocess Data

### Use just first two tags

In [2]:
def just_two_tags(examples):
    tags = [tag[:2] for tag in examples['Tags']]
    examples['Tags'] = tags
    return examples

ds = ds.map(just_two_tags, batched=True)

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()
labels = mlb.fit(ds['train']['Tags'])  # Fit all unique Tags

# Save label mappings
import json
with open("label_mapping.json", "w") as f:
    json.dump(mlb.classes_.tolist(), f)


In [4]:
def preprocess(examples):
    tokens = tokenizer(examples["Tibetan"], padding="max_length", truncation=True, max_length=128)
    tokens["labels"] =  mlb.transform(examples['Tags']).astype(float).tolist() # Convert labels to multi-hot
    return tokens

encoded_dataset = ds.map(preprocess, batched=True)


In [5]:
encoded_dataset = encoded_dataset.remove_columns(['Tibetan', 'Phonetic', 'English', 'Tags'])

In [6]:
encoded_dataset = encoded_dataset['train'].train_test_split(.15)

In [7]:
enc = tokenizer(ds['train'][0]['Tibetan'])
tokenizer.decode(enc.input_ids)

'[CLS] བ ་ མ ་ དང ་ ལག ་ པའ ་ ལ ་ ལ ་ ཕག ་ འཚལ ་ ལ ། ། [SEP]'

In [8]:
ds['train'][0]

{'Tibetan': 'བླ་མ་དང་ལྷག་པའི་ལྷ་ལ་ཕྱག་འཚལ་ལོ།།',
 'Phonetic': 'lama dang lhakpé lha la chaktsal lo',
 'English': 'Homage to the guru and supreme deity.',
 'Tags': ['Abhidharma', 'Praise']}

## Train Model

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load tokenizer and model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(mlb.classes_))

# Resize embeddings to match the new tokenizer
model.resize_token_embeddings(len(tokenizer))

# Move model to GPU
model = model.to('cuda:0')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    predictions, references = eval_pred
    
    # Apply a threshold to convert logits to binary predictions
    predictions = (predictions > 0.5).astype(int)
    
    # Compute metrics
    accuracy = accuracy_score(references, predictions)
    f1 = f1_score(references, predictions, average="micro")
    precision = precision_score(references, predictions, average="micro")
    recall = recall_score(references, predictions, average="micro")

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision_1": precision,
        "recall": recall,
    }


In [11]:
from transformers import TrainingArguments, Trainer

from transformers import EarlyStoppingCallback

# Define training arguments
training_args = TrainingArguments(
    output_dir="bert-classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,  # Set a maximum number of epochs
    weight_decay=0.01,
    eval_strategy="epoch",  # Evaluate at the end of every epoch
    save_strategy="epoch",  # Save the model at the end of every epoch
    load_best_model_at_end=True,  # Load the best model after training
    metric_for_best_model="accuracy",  # Metric to monitor
    greater_is_better=True,  # Higher accuracy is better
    logging_dir="./logs"
)

# Add the EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3  # Stop training if the metric does not improve for 3 evaluation steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]  # Add the early stopping callback
)

# Start training
trainer.train()

/tmp/ipykernel_9135/3546473095.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


  0%|          | 0/571300 [00:00<?, ?it/s]

{'loss': 0.1586, 'grad_norm': 0.06657019257545471, 'learning_rate': 1.9982496061613864e-05, 'epoch': 0.09}
{'loss': 0.0555, 'grad_norm': 0.06579519808292389, 'learning_rate': 1.996499212322773e-05, 'epoch': 0.18}
{'loss': 0.0526, 'grad_norm': 0.0541217103600502, 'learning_rate': 1.994748818484159e-05, 'epoch': 0.26}
{'loss': 0.0522, 'grad_norm': 0.05693612992763519, 'learning_rate': 1.9929984246455454e-05, 'epoch': 0.35}
{'loss': 0.0519, 'grad_norm': 0.06708614528179169, 'learning_rate': 1.9912480308069317e-05, 'epoch': 0.44}
{'loss': 0.0516, 'grad_norm': 0.0570235550403595, 'learning_rate': 1.9894976369683182e-05, 'epoch': 0.53}
{'loss': 0.0517, 'grad_norm': 0.05111713334918022, 'learning_rate': 1.987747243129704e-05, 'epoch': 0.61}
{'loss': 0.0515, 'grad_norm': 0.054363153874874115, 'learning_rate': 1.9859968492910907e-05, 'epoch': 0.7}
{'loss': 0.0516, 'grad_norm': 0.048229996114969254, 'learning_rate': 1.984246455452477e-05, 'epoch': 0.79}
{'loss': 0.0515, 'grad_norm': 0.0492477044

  0%|          | 0/1009 [00:00<?, ?it/s]

/home/j/Desktop/MLotsawa/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

  0%|          | 0/1009 [00:00<?, ?it/s]

{'eval_loss': 0.0148966358974576, 'eval_accuracy': 0.7777915555831112, 'eval_f1': 0.8579094538460296, 'eval_precision_1': 0.8962547701867549, 'eval_recall': 0.8227106454212908, 'eval_runtime': 40.0029, 'eval_samples_per_second': 403.195, 'eval_steps_per_second': 25.223, 'epoch': 61.0}
{'loss': 0.0028, 'grad_norm': 0.07926646620035172, 'learning_rate': 7.799754944862595e-06, 'epoch': 61.0}
{'loss': 0.0027, 'grad_norm': 0.7245857119560242, 'learning_rate': 7.782251006476457e-06, 'epoch': 61.09}
{'loss': 0.0029, 'grad_norm': 0.07150136679410934, 'learning_rate': 7.764747068090321e-06, 'epoch': 61.18}
{'loss': 0.0027, 'grad_norm': 0.06456363201141357, 'learning_rate': 7.747243129704184e-06, 'epoch': 61.26}
{'loss': 0.0028, 'grad_norm': 0.026240836828947067, 'learning_rate': 7.729739191318048e-06, 'epoch': 61.35}
{'loss': 0.003, 'grad_norm': 0.0442662313580513, 'learning_rate': 7.71223525293191e-06, 'epoch': 61.44}
{'loss': 0.0029, 'grad_norm': 0.11813411116600037, 'learning_rate': 7.694731

  0%|          | 0/1009 [00:00<?, ?it/s]

{'eval_loss': 0.01483156718313694, 'eval_accuracy': 0.7746915493830988, 'eval_f1': 0.8579020707087074, 'eval_precision_1': 0.8986422267481331, 'eval_recall': 0.8206956413912828, 'eval_runtime': 40.035, 'eval_samples_per_second': 402.872, 'eval_steps_per_second': 25.203, 'epoch': 62.0}
{'train_runtime': 43526.8189, 'train_samples_per_second': 209.976, 'train_steps_per_second': 13.125, 'train_loss': 0.013663096298290512, 'epoch': 62.0}


TrainOutput(global_step=354206, training_loss=0.013663096298290512, metrics={'train_runtime': 43526.8189, 'train_samples_per_second': 209.976, 'train_steps_per_second': 13.125, 'total_flos': 3.732920048296366e+17, 'train_loss': 0.013663096298290512, 'epoch': 62.0})

## Test Outputs

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import json

# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("/home/j/Desktop/MLotsawa/Notebooks/Models/BertTag/bert-classifier/checkpoint-34275")
tokenizer = BertTokenizer.from_pretrained("./tibetan_tokenizer")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Load label mapping
with open("label_mapping.json", "r") as f:
    label_mapping = json.load(f)

# Example input text
input_text = "བླ་མ་དང་ལྷག་པའི་ལྷ་ལ་ཕྱག་འཚལ་ལོ།།"

# Tokenize input
encoded_input = tokenizer(
    input_text, 
    padding="max_length", 
    truncation=True, 
    max_length=128, 
    return_tensors="pt"
)
encoded_input = {key: val.to(device) for key, val in encoded_input.items()}

# Get predictions
with torch.no_grad():
    outputs = model(**encoded_input)
    logits = outputs.logits
    probabilities = torch.sigmoid(logits).cpu().numpy()
    predictions = (probabilities > 0.5).astype(int)

# Decode predictions
predicted_tags = [label_mapping[i] for i, val in enumerate(predictions[0]) if val == 1]

print("Predicted Tags:", predicted_tags)


OSError: Incorrect path_or_model_id: '/home/j/Desktop/MLotsawa/Notebooks/Models/BertTag/bert-classifier/checkpoint-34275'. Please provide either the path to a local folder or the repo_id of a model on the Hub.